<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Tradevesting/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime
import pytz
tz_NY = pytz.timezone('Asia/Kolkata')
datetime_NY = datetime.datetime.now(tz_NY)
print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

gen_output = 0

Run date time (IST): 2023-07-27 08:46:59


In [2]:
# install libraries
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

In [3]:
# import libraries

%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib
yfin.pdr_override()
import pandas_ta as ta

In [4]:
# set start and end date
start_date = (datetime.date.today() + datetime.timedelta(days=-5*365)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')



In [5]:
# fetch nifty200 scrips

import pandas as pd
import numpy as np

mypf = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Data-Science-with-Python/master/Tradevesting/myProspectsScrips.csv')

myts = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Data-Science-with-Python/master/Tradevesting/myTransactionsReport.csv')

mypf = mypf[mypf['InPortfolio'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol'].unique()

stock_ids = stock_n100

#mypf = mypf[mypf.Forecast.notnull()]
#stock_ids = mypf['Symbol'].unique()

stock_ids.sort()
stock_ids

array(['5PAISA', 'AAVAS', 'AMARAJABAT', 'AMBUJACEM', 'BAJAJFINSV',
       'BAJAJHIND', 'BANDHANBNK', 'BATAINDIA', 'BAYERCROP', 'CAMS',
       'CDSL', 'CLEAN', 'DMART', 'EICHERMOT', 'EMAMILTD', 'FINEORG',
       'GICRE', 'GILLETTE', 'GLAND', 'HDFCAMC', 'HDFCBANK', 'ICICIGI',
       'INDIGOPNTS', 'INDUSINDBK', 'INFY', 'IRCTC', 'ITBEES', 'J&KBANK',
       'JMFINANCIL', 'JUBLFOOD', 'KOTAKBANK', 'LTIM', 'LUXIND',
       'METROPOLIS', 'NIACL', 'NMDC', 'PAGEIND', 'PIDILITIND', 'PNB',
       'RAJESHEXPO', 'RELAXO', 'ROSSARI', 'SAIL', 'SRF', 'STARHEALTH',
       'SUNTV', 'TCS', 'TEAMLEASE', 'UJJIVANSFB', 'VAIBHAVGBL',
       'VALIANTORG', 'WIPRO'], dtype=object)

In [6]:
# function to compute stock attributes
def get_common_features():
  def stock_prec_dev(stock_symbol):
      #stock_symbol = 'ULTRACEMCO.NS'
      short_window = 20
      mid_window = 50
      long_window = 200
      moving_avg = 'SMA'

      # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
      start = datetime.datetime(*map(int, start_date.split('-')))
      end = datetime.datetime(*map(int, end_date.split('-')))
      stock_df = yfin.Ticker(stock_symbol).history(period='7y')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
      stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
      stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
      stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
      stock_df.dropna(axis = 0, inplace = True) # remove any null rows

      stock_df['Stock'] = stock_symbol

      stock_df['Close'] = round(stock_df['Close'],2)
      stock_df['Max'] = round(max(stock_df['Close']),0)
      stock_df.ta.rsi(append=True)
      #stock_df.rename(columns={'RSI_14':'RSI'}, inplace=True)
      stock_df['Close'] = round(stock_df['Close'],0)
      stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
      stock_df.drop(['Open', 'Low', 'High', 'Volume'
                    ], axis=1, inplace=True)

      return stock_df

  df_prec_dev = pd.DataFrame()

  #tmp = stock_prec_dev('ULTRACEMCO.NS')
  #df_perform = df_perform.append(tmp)
  for stock_id in stock_ids:
      #print(stock_id)
      #current_pe = get_current_pe(stock_id)
      stock_id = stock_id.upper() + '.NS'
      tmp = stock_prec_dev(stock_id)
      tmp['Prev_Close'] = tmp['Close'].shift(1)
      tmp = tmp.reset_index()
      tmp = tmp.tail(1)
      tmp['Avg_Cost'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['AvgCost'].values[0]
      tmp['Shares'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Shares'].values[0]
      tmp['Criteria'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Criteria'].values[0]
      tmp['Strategy'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Strategy'].values[0]
      tmp['Target'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Target'].values[0]
      tmp['LatestQtr'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['LatestQtr'].values[0]
      tmp['StarStock'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['StarStock'].values[0]
      tmp['Conviction'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Conviction'].values[0]
      tmp['Category'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Category'].values[0]
      #tmp['Curr_PE'] = current_pe
      df_prec_dev = df_prec_dev.append(tmp)

  return df_prec_dev

df_common_features = get_common_features()

In [7]:
df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)

In [8]:
# portfolio stocks

def get_portfolio_features(df_common_features):

  df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])

  tmp_df = df_common_features[df_common_features['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
  print('qualified stocks: '+str(tmp_df.shape[0]))
  tmp_df['Investment'] = tmp_df['Avg_Cost'] * tmp_df['Shares']
  tmp_df['Current'] = tmp_df['Close'] * tmp_df['Shares']
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['Estimated'] = tmp_df['Target'] * tmp_df['Shares']
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['Estimated P/L%'] = round((tmp_df['Estimated'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['Juice Left%'] = tmp_df['Estimated P/L%'] - tmp_df['Current P/L%']
  investment = round(sum(tmp_df['Avg_Cost']*tmp_df['Shares']),0)
  tmp_df['Weightage%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['Weightage%'],0)

  return tmp_df

df_portfolio_features = get_portfolio_features(df_common_features)

def print_portfolio_stats(df_portfolio_features):
  investment = round(sum(df_portfolio_features['Avg_Cost']*df_portfolio_features['Shares']),0)
  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0)
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0)

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-investment
  curr_pnl_percentage = round((current-investment)*100/investment,2)

  estimate = round(sum(df_portfolio_features['Max']*df_portfolio_features['Shares']),0)
  est_pnl_amount = estimate-investment
  est_pnl_percentage = round((estimate-investment)*100/investment,2)

  print('Investment: ', investment)
  print('Current: ', current)
  print('Today PnL: ', today_pnl_amount)
  print('Today PnL %: ', today_pnl_percentage)
  print('Current PnL: ', curr_pnl_amount)
  print('Current PnL %: ', curr_pnl_percentage)
  print('Estimate: ', estimate)
  print('Estimate PnL: ', est_pnl_amount)
  print('Estimate PnL %: ', est_pnl_percentage)
  print('-------------------')
  print('Total profit: ',round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0))
  print('Total loss: ',round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0))

print_portfolio_stats(df_portfolio_features)

qualified stocks: 52
Investment:  3443229.0
Current:  3634959.0
Today PnL:  10380.0
Today PnL %:  0.29
Current PnL:  191730.0
Current PnL %:  5.57
Estimate:  6104283.0
Estimate PnL:  2661054.0
Estimate PnL %:  77.28
-------------------
Total profit:  231092.0
Total loss:  -39362.0


In [9]:
cols = ['Stock', 'Close', 'Dev%_200', 'RSI_14', 'Target', 'Avg_Cost', 'Shares', 'Today P/L%', 'Current P/L%', 'Estimated P/L%', 'Juice Left%', 'Weightage%', 'Risk Ind', 'LatestQtr', 'StarStock', 'Criteria', 'Strategy', 'Conviction', 'Category']
df_portfolio_features = df_portfolio_features[cols]

if gen_output == 1:
  from google.colab import drive
  drive.mount('/content/drive')
  df_portfolio_features.to_csv('/content/drive/My Drive/data/stocks/myStocks-Portfolio-Analysis.csv', index=False)

In [10]:
df_portfolio_features.sort_values(by = 'Current P/L%', ascending=False)

,Stock,Close,Dev%_200,RSI_14,Target,Avg_Cost,Shares,Today P/L%,Current P/L%,Estimated P/L%,Juice Left%,Weightage%,Risk Ind,LatestQtr,StarStock,Criteria,Strategy,Conviction,Category
0,5PAISA.NS,464.0,43.68,75.0,532.0,347.06,228.0,2.20,33.69,53.29,19.60,2.30,77.0,1,1.0,V40N,ATH,M,FINANCE
19,HDFCAMC.NS,2486.0,27.24,73.0,3250.0,1973.85,50.0,-1.62,25.95,64.65,38.70,2.87,74.0,1,0.0,V40,ATH,H,FINANCE
17,GILLETTE.NS,5552.0,15.12,66.0,7750.0,4475.85,21.0,0.73,24.04,73.15,49.11,2.73,66.0,0,0.0,V40,ATH,H,FMCG
48,UJJIVANSFB.NS,43.0,42.17,63.0,55.0,36.90,1850.0,-2.27,16.53,49.05,32.52,1.98,33.0,0,1.0,V40N,ATH,M,BANKS
21,ICICIGI.NS,1369.0,17.03,60.0,1600.0,1177.58,84.0,-0.29,16.26,35.87,19.61,2.87,47.0,1,1.0,V40,ATH,H,INSURANCE
16,GICRE.NS,202.0,26.13,74.0,370.0,174.18,231.0,0.00,15.97,112.42,96.45,1.17,19.0,0,0.0,VR,2T3Y,L,INSURANCE
38,PNB.NS,63.0,26.30,66.0,150.0,54.97,1400.0,3.28,14.61,172.88,158.27,2.24,33.0,0,1.0,VR,ATH,M,BANKS
45,SUNTV.NS,541.0,16.30,80.0,900.0,473.49,108.0,0.56,14.26,90.08,75.82,1.49,21.0,0,1.0,V40N,ATH,M,ENTERTAINMENT
7,BATAINDIA.NS,1707.0,7.10,65.0,2000.0,1523.62,100.0,0.35,12.04,31.27,19.23,4.42,53.0,0,0.0,V40,ATH,H,FOOTWEAR
4,BAJAJFINSV.NS,1626.0,9.60,62.0,1800.0,1454.93,52.0,-1.39,11.76,23.72,11.96,2.20,26.0,0,1.0,V40,ATH,H,FINANCE


In [11]:
df_portfolio_features.sort_values(by = 'Juice Left%', ascending=True)

,Stock,Close,Dev%_200,RSI_14,Target,Avg_Cost,Shares,Today P/L%,Current P/L%,Estimated P/L%,Juice Left%,Weightage%,Risk Ind,LatestQtr,StarStock,Criteria,Strategy,Conviction,Category
37,PIDILITIND.NS,2607.0,3.74,48.0,2780.0,2504.06,30.0,0.42,4.11,11.02,6.91,2.18,9.0,0,1.0,V40,ATH,H,CHEMICALS
13,EICHERMOT.NS,3354.0,0.14,47.0,3700.0,3260.20,15.0,0.84,2.88,13.49,10.61,1.42,4.0,0,1.0,V40N,ATH,H,AUTO
4,BAJAJFINSV.NS,1626.0,9.60,62.0,1800.0,1454.93,52.0,-1.39,11.76,23.72,11.96,2.20,26.0,0,1.0,V40,ATH,H,FINANCE
46,TCS.NS,3388.0,4.71,56.0,3900.0,3231.00,16.0,-0.32,4.86,20.71,15.85,1.50,7.0,1,1.0,V40,ATH,H,IT
5,BAJAJHIND.NS,19.0,32.86,66.0,22.0,17.15,1750.0,0.00,10.79,28.28,17.49,0.87,9.0,0,1.0,VR,ATH,M,SUGAR
30,KOTAKBANK.NS,1895.0,2.84,52.0,2270.0,1955.95,26.0,1.01,-3.12,16.06,19.18,1.48,-5.0,1,1.0,V40,BTT,L,BANKS
7,BATAINDIA.NS,1707.0,7.10,65.0,2000.0,1523.62,100.0,0.35,12.04,31.27,19.23,4.42,53.0,0,0.0,V40,ATH,H,FOOTWEAR
0,5PAISA.NS,464.0,43.68,75.0,532.0,347.06,228.0,2.20,33.69,53.29,19.60,2.30,77.0,1,1.0,V40N,ATH,M,FINANCE
21,ICICIGI.NS,1369.0,17.03,60.0,1600.0,1177.58,84.0,-0.29,16.26,35.87,19.61,2.87,47.0,1,1.0,V40,ATH,H,INSURANCE
20,HDFCBANK.NS,1691.0,5.80,59.0,2050.0,1683.55,30.0,-0.35,0.44,21.77,21.33,1.47,1.0,1,1.0,V40,BTT,H,BANKS


In [12]:
df_portfolio_features.sort_values(by = 'RSI_14', ascending=True)

,Stock,Close,Dev%_200,RSI_14,Target,Avg_Cost,Shares,Today P/L%,Current P/L%,Estimated P/L%,Juice Left%,Weightage%,Risk Ind,LatestQtr,StarStock,Criteria,Strategy,Conviction,Category
11,CLEAN.NS,1295.0,-10.63,30.0,2500.0,1365.11,39.0,-0.15,-5.14,83.14,88.28,1.55,-8.0,0,1.0,VR,2T3Y,L,CHEMICALS
43,SRF.NS,2144.0,-8.99,34.0,2666.0,2222.00,14.0,-0.92,-3.51,19.98,23.49,0.90,-3.0,1,0.0,VR,ATH,M,CHEMICALS
39,RAJESHEXPO.NS,512.0,-22.76,38.0,835.0,578.92,87.0,-0.19,-11.56,44.23,55.79,1.46,-17.0,0,1.0,V40N,ATH,M,JEWELLERY
8,BAYERCROP.NS,4399.0,-0.59,39.0,6050.0,4346.57,16.0,-2.00,1.21,39.19,37.98,2.02,2.0,0,1.0,V40N,ATH,M,CHEMICALS
29,JUBLFOOD.NS,468.0,-5.46,41.0,800.0,471.56,164.0,-1.89,-0.75,69.65,70.40,2.25,-2.0,1,0.0,AR,ATH,M,RESTAURANTS
50,VALIANTORG.NS,506.0,-7.34,41.0,1550.0,532.61,185.0,0.00,-5.00,191.02,196.02,2.86,-14.0,0,0.0,VR,2T3Y,L,CHEMICALS
25,IRCTC.NS,620.0,-4.62,42.0,900.0,633.05,120.0,0.16,-2.06,42.17,44.23,2.21,-5.0,0,1.0,AR,ATH,M,TRAVEL
6,BANDHANBNK.NS,221.0,-6.48,42.0,600.0,244.52,400.0,1.38,-9.62,145.38,155.00,2.84,-27.0,1,0.0,VR,2T3Y,M,BANKS
31,LTIM.NS,4861.0,4.64,43.0,7000.0,4987.59,10.0,-0.16,-2.54,40.35,42.89,1.45,-4.0,1,1.0,V200,ATH,M,IT
2,AMARAJABAT.NS,630.0,5.48,43.0,1010.0,625.25,72.0,0.80,0.76,61.54,60.78,1.31,1.0,0,1.0,V200,ATH,M,AUTO


In [13]:
df_portfolio_features.sort_values(by = 'Estimated P/L%', ascending=False)

,Stock,Close,Dev%_200,RSI_14,Target,Avg_Cost,Shares,Today P/L%,Current P/L%,Estimated P/L%,Juice Left%,Weightage%,Risk Ind,LatestQtr,StarStock,Criteria,Strategy,Conviction,Category
18,GLAND.NS,1195.0,-15.16,64.0,3900.0,1225.87,55.0,-2.61,-2.52,218.14,220.66,1.96,-5.0,0,0.0,VR,2T3Y,L,PHARMA
50,VALIANTORG.NS,506.0,-7.34,41.0,1550.0,532.61,185.0,0.00,-5.00,191.02,196.02,2.86,-14.0,0,0.0,VR,2T3Y,L,CHEMICALS
34,NIACL.NS,122.0,11.09,60.0,333.0,117.07,390.0,-1.61,4.21,184.45,180.24,1.33,6.0,0,0.0,VR,ATH,L,INSURANCE
32,LUXIND.NS,1558.0,3.47,65.0,4200.0,1493.72,51.0,4.42,4.30,181.18,176.88,2.21,10.0,0,0.0,V40N,2T3Y,M,TEXTILES
49,VAIBHAVGBL.NS,340.0,7.74,67.0,875.0,317.48,144.0,-0.29,7.09,175.61,168.52,1.33,9.0,0,0.0,VR,2T3Y,L,JEWELLERY
38,PNB.NS,63.0,26.30,66.0,150.0,54.97,1400.0,3.28,14.61,172.88,158.27,2.24,33.0,0,1.0,VR,ATH,M,BANKS
6,BANDHANBNK.NS,221.0,-6.48,42.0,600.0,244.52,400.0,1.38,-9.62,145.38,155.00,2.84,-27.0,1,0.0,VR,2T3Y,M,BANKS
28,JMFINANCIL.NS,74.0,8.97,55.0,160.0,69.17,675.0,2.78,6.98,131.31,124.33,1.36,9.0,0,0.0,VR,2T3Y,L,FINANCE
33,METROPOLIS.NS,1395.0,1.84,45.0,3100.0,1365.98,22.0,-0.57,2.12,126.94,124.82,0.87,2.0,0,0.0,VR,ATH,L,HEALTHCARE
16,GICRE.NS,202.0,26.13,74.0,370.0,174.18,231.0,0.00,15.97,112.42,96.45,1.17,19.0,0,0.0,VR,2T3Y,L,INSURANCE


In [14]:
df_portfolio_features.sort_values(by = 'Weightage%', ascending=False)

,Stock,Close,Dev%_200,RSI_14,Target,Avg_Cost,Shares,Today P/L%,Current P/L%,Estimated P/L%,Juice Left%,Weightage%,Risk Ind,LatestQtr,StarStock,Criteria,Strategy,Conviction,Category
22,INDIGOPNTS.NS,1513.0,17.26,70.0,2600.0,1407.73,124.0,0.73,7.48,84.69,77.21,5.07,38.0,0,1.0,V40N,2T3Y,M,PAINTS
36,PAGEIND.NS,37673.0,-9.37,55.0,50000.0,38548.77,4.0,1.28,-2.27,29.71,31.98,4.48,-10.0,0,1.0,V40,ATH,H,APPARELS
7,BATAINDIA.NS,1707.0,7.10,65.0,2000.0,1523.62,100.0,0.35,12.04,31.27,19.23,4.42,53.0,0,0.0,V40,ATH,H,FOOTWEAR
24,INFY.NS,1349.0,-4.20,50.0,1800.0,1324.07,104.0,1.05,1.88,35.94,34.06,4.00,8.0,1,1.0,V40,ATH,H,IT
26,ITBEES.NS,31.0,4.57,55.0,40.0,31.49,3350.0,0.00,-1.56,27.02,28.58,3.06,-5.0,0,1.0,AR,ATH,L,IT
21,ICICIGI.NS,1369.0,17.03,60.0,1600.0,1177.58,84.0,-0.29,16.26,35.87,19.61,2.87,47.0,1,1.0,V40,ATH,H,INSURANCE
19,HDFCAMC.NS,2486.0,27.24,73.0,3250.0,1973.85,50.0,-1.62,25.95,64.65,38.70,2.87,74.0,1,0.0,V40,ATH,H,FINANCE
50,VALIANTORG.NS,506.0,-7.34,41.0,1550.0,532.61,185.0,0.00,-5.00,191.02,196.02,2.86,-14.0,0,0.0,VR,2T3Y,L,CHEMICALS
6,BANDHANBNK.NS,221.0,-6.48,42.0,600.0,244.52,400.0,1.38,-9.62,145.38,155.00,2.84,-27.0,1,0.0,VR,2T3Y,M,BANKS
17,GILLETTE.NS,5552.0,15.12,66.0,7750.0,4475.85,21.0,0.73,24.04,73.15,49.11,2.73,66.0,0,0.0,V40,ATH,H,FMCG


In [15]:
df_portfolio_features.sort_values(by = 'Risk Ind', ascending=True)

,Stock,Close,Dev%_200,RSI_14,Target,Avg_Cost,Shares,Today P/L%,Current P/L%,Estimated P/L%,Juice Left%,Weightage%,Risk Ind,LatestQtr,StarStock,Criteria,Strategy,Conviction,Category
6,BANDHANBNK.NS,221.0,-6.48,42.0,600.0,244.52,400.0,1.38,-9.62,145.38,155.00,2.84,-27.0,1,0.0,VR,2T3Y,M,BANKS
39,RAJESHEXPO.NS,512.0,-22.76,38.0,835.0,578.92,87.0,-0.19,-11.56,44.23,55.79,1.46,-17.0,0,1.0,V40N,ATH,M,JEWELLERY
50,VALIANTORG.NS,506.0,-7.34,41.0,1550.0,532.61,185.0,0.00,-5.00,191.02,196.02,2.86,-14.0,0,0.0,VR,2T3Y,L,CHEMICALS
36,PAGEIND.NS,37673.0,-9.37,55.0,50000.0,38548.77,4.0,1.28,-2.27,29.71,31.98,4.48,-10.0,0,1.0,V40,ATH,H,APPARELS
35,NMDC.NS,112.0,-0.53,64.0,140.0,122.55,265.0,0.00,-8.61,14.24,22.85,0.94,-8.0,0,0.0,VR,ATH,M,MINING
11,CLEAN.NS,1295.0,-10.63,30.0,2500.0,1365.11,39.0,-0.15,-5.14,83.14,88.28,1.55,-8.0,0,1.0,VR,2T3Y,L,CHEMICALS
25,IRCTC.NS,620.0,-4.62,42.0,900.0,633.05,120.0,0.16,-2.06,42.17,44.23,2.21,-5.0,0,1.0,AR,ATH,M,TRAVEL
26,ITBEES.NS,31.0,4.57,55.0,40.0,31.49,3350.0,0.00,-1.56,27.02,28.58,3.06,-5.0,0,1.0,AR,ATH,L,IT
30,KOTAKBANK.NS,1895.0,2.84,52.0,2270.0,1955.95,26.0,1.01,-3.12,16.06,19.18,1.48,-5.0,1,1.0,V40,BTT,L,BANKS
18,GLAND.NS,1195.0,-15.16,64.0,3900.0,1225.87,55.0,-2.61,-2.52,218.14,220.66,1.96,-5.0,0,0.0,VR,2T3Y,L,PHARMA


In [16]:
# SIP based on portfolio
df_portfolio_features[(df_portfolio_features['Weightage%'] < 2) & (df_portfolio_features['Dev%_200'] < 0)].sort_values(by = 'RSI_14', ascending=True)

,Stock,Close,Dev%_200,RSI_14,Target,Avg_Cost,Shares,Today P/L%,Current P/L%,Estimated P/L%,Juice Left%,Weightage%,Risk Ind,LatestQtr,StarStock,Criteria,Strategy,Conviction,Category
11,CLEAN.NS,1295.0,-10.63,30.0,2500.0,1365.11,39.0,-0.15,-5.14,83.14,88.28,1.55,-8.0,0,1.0,VR,2T3Y,L,CHEMICALS
43,SRF.NS,2144.0,-8.99,34.0,2666.0,2222.00,14.0,-0.92,-3.51,19.98,23.49,0.90,-3.0,1,0.0,VR,ATH,M,CHEMICALS
39,RAJESHEXPO.NS,512.0,-22.76,38.0,835.0,578.92,87.0,-0.19,-11.56,44.23,55.79,1.46,-17.0,0,1.0,V40N,ATH,M,JEWELLERY
12,DMART.NS,3713.0,-0.94,47.0,5100.0,3659.46,14.0,0.98,1.46,39.36,37.90,1.49,2.0,1,1.0,AR,ATH,M,FMCG
15,FINEORG.NS,4717.0,-8.73,47.0,7000.0,4697.43,11.0,1.09,0.42,49.02,48.60,1.50,1.0,0,1.0,V40N,ATH,M,CHEMICALS
47,TEAMLEASE.NS,2379.0,-2.81,47.0,4550.0,2279.93,21.0,-1.41,4.35,99.57,95.22,1.39,6.0,0,1.0,V40N,2T3Y,M,MISC
18,GLAND.NS,1195.0,-15.16,64.0,3900.0,1225.87,55.0,-2.61,-2.52,218.14,220.66,1.96,-5.0,0,0.0,VR,2T3Y,L,PHARMA
35,NMDC.NS,112.0,-0.53,64.0,140.0,122.55,265.0,0.00,-8.61,14.24,22.85,0.94,-8.0,0,0.0,VR,ATH,M,MINING
1,AAVAS.NS,1616.0,-7.17,70.0,3075.0,1453.92,37.0,-0.19,11.15,111.50,100.35,1.56,17.0,0,1.0,VR,2T3Y,M,FINANCE


In [17]:
tmp_df = df_portfolio_features.groupby('Category')['Weightage%'].agg(['sum']).reset_index()
tmp_df = tmp_df.rename({'sum': 'Weight%'}, axis='columns')
tmp_df.sort_values(by = 'Weight%', ascending=False)

,Category,Weight%
2,BANKS,12.24
11,IT,12.21
4,CHEMICALS,11.89
6,FINANCE,10.29
8,FOOTWEAR,6.61
10,INSURANCE,6.24
7,FMCG,5.10
15,PAINTS,5.07
0,APPARELS,4.48
14,MISC,4.47


In [18]:
tmp_df = df_portfolio_features.groupby('Criteria')['Weightage%'].agg(['sum']).reset_index()
tmp_df = tmp_df.rename({'sum': 'Weight%'}, axis='columns')
tmp_df.sort_values(by = 'Weight%', ascending=False)

,Criteria,Weight%
4,VR,34.13
2,V40,30.20
3,V40N,23.03
0,AR,9.01
1,V200,3.64


In [19]:
tmp_df = df_portfolio_features.groupby('Strategy')['Weightage%'].agg(['sum']).reset_index()
tmp_df = tmp_df.rename({'sum': 'Weight%'}, axis='columns')
tmp_df.sort_values(by = 'Weight%', ascending=False)

,Strategy,Weight%
1,ATH,68.49
0,2T3Y,28.57
2,BTT,2.95


In [20]:
tmp_df = df_portfolio_features.groupby('Conviction')['Weightage%'].agg(['sum']).reset_index()
tmp_df = tmp_df.rename({'sum': 'Weight%'}, axis='columns')
tmp_df.sort_values(by = 'Weight%', ascending=False)

,Conviction,Weight%
2,M,45.40
0,H,30.14
1,L,24.47


In [21]:
#myts['Value'] = myts['Value'].str.replace(',', '').astype(float)

In [22]:
#sum(myts[(myts['Type'] == 'Buy')]['Value'])

In [23]:
#sum(myts[(myts['Type'] == 'Sell')]['Value'])